In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPooling3D, UpSampling2D, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import backend as K

2023-10-15 04:36:08.214437: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-15 04:36:08.260582: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 04:36:09.081598: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
X_train = np.load('../y_smp_train.npy')
y_train = np.load('../pars_smp_train.npy')
X_test = np.load('../y_smp_test.npy')

In [3]:
X_train.shape, y_train.shape

((100000, 200, 3), (100000, 15, 1))

In [4]:
#!g2.1
def quantile_loss_75(y_true, y_pred):
    error = y_true - y_pred
    loss = K.mean(K.maximum(0.75 * error, (0.75 - 1) * error))
    return loss

In [5]:
#!g2.1
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(200, 3, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=quantile_loss_75, optimizer=opt, metrics=[RootMeanSquaredError()])

2023-10-15 04:36:48.912701: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 04:36:49.743538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-15 04:40:03.788335: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78980 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:8c:00.0, compute capability: 8.0


In [6]:
#!g2.1
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 3, 16)        160       
                                                                 
 up_sampling2d (UpSampling2D  (None, 400, 6, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 400, 6, 32)        4640      
                                                                 
 up_sampling2d_1 (UpSampling  (None, 800, 12, 32)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 132, 7, 32)        73760     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 66, 4, 32)        0

In [7]:
#!g2.1
model.fit(X_train, y_train, batch_size=128, epochs=25)

Epoch 1/25


2023-10-15 04:40:23.182003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-10-15 04:40:23.883598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-15 04:40:23.924872: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f7af3e2d240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-15 04:40:23.924899: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-10-15 04:40:23.929188: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-15 04:40:24.052561: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

782/782 [==============================] - 13s 9ms/step - loss: 0.2470 - root_mean_squared_error: 0.8202
Epoch 2/25
782/782 [==============================] - 7s 9ms/step - loss: 0.1038 - root_mean_squared_error: 0.4701
Epoch 3/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0878 - root_mean_squared_error: 0.4004
Epoch 4/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0802 - root_mean_squared_error: 0.3693
Epoch 5/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0757 - root_mean_squared_error: 0.3508
Epoch 6/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0731 - root_mean_squared_error: 0.3397
Epoch 7/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0708 - root_mean_squared_error: 0.3306
Epoch 8/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0689 - root_mean_squared_error: 0.3219
Epoch 9/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0672 - root_me

KeyboardInterrupt: 

In [ ]:
#!g2.1
pred = model.predict(X_test)

In [ ]:
#!g2.1
np.save('test_pred_75', pred)

In [ ]:
#!g2.1
model.save("model_75.h5")

In [ ]:
#!g2.1
